In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np

In [ ]:
df = pd.read_excel('default_of_credit_card_clients__courseware_version_1_21_19.xls')

In [ ]:
df.shape

In [46]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,EDUCATION_CAT,graduate school,high school,others,university
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,1,university,False,False,False,True
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,1000,1000,0,2000,1,university,False,False,False,True
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,1000,1000,1000,5000,0,university,False,False,False,True
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,1200,1100,1069,1000,0,university,False,False,False,True
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,10000,9000,689,679,0,university,False,False,False,True


In [ ]:
df['ID'].nunique()

Listar os IDs exclusivos e a medida que ocorrem

In [ ]:
id_counts = df['ID'].value_counts()
id_counts.head()

Nenhum id aparece mais de duas vezes dentro do dataset


In [ ]:
id_counts.value_counts()

Localizando duplicadas por meio de uma máscara lógica para armazenar valores booleanos

In [ ]:
dupe_mask = id_counts == 2
dupe_mask[:5]

Valores em lista que estão duplicados

In [ ]:
dupe_ids = id_counts.index[dupe_mask]
dupe_ids = list(dupe_ids)
len(dupe_ids)

In [ ]:
dupe_ids[0:5]

Partes duplicadas do dataset

In [ ]:
df.loc[df['ID'].isin(dupe_ids[:3]),:].head(10)

Número de IDs duplicados  para cada coluna exceto a primeira

In [ ]:
df_zero_mask = df == 0
feature_zero_mask = df_zero_mask.iloc[:,1:].all(axis=1)
sum(feature_zero_mask)

Limpando o dataset e criando uma cópia

In [ ]:
df_clean_1 = df.loc[~feature_zero_mask,:].copy()
df_clean_1.shape

Valores únicos depois da mudança

In [ ]:
df_clean_1['ID'].nunique()

Podemos observar que não existem dados ausentes

In [ ]:
df_clean_1.info()

De acordo com os valores do dicionário de dados, é esperado que só tenha valores int

In [ ]:
df_clean_1['PAY_1'].head()

A saída apresenta os valores de -2 e 0 que não foram documentados e uma coluna de dados ausentes

In [ ]:
df_clean_1['PAY_1'].value_counts()

Retirando os valores ausentes

In [ ]:
valid_pay_1_mask = df_clean_1['PAY_1'] != 'Not available'
valid_pay_1_mask[:5]

In [ ]:
sum(valid_pay_1_mask)

In [ ]:
df_clean_2 = df_clean_1.loc[valid_pay_1_mask,:].copy()

In [ ]:
df_clean_2.shape

Convertendo para int os valores da coluna PAY_1

In [ ]:
df_clean_2['PAY_1'] = df_clean_2['PAY_1'].astype('int64')

In [ ]:
df_clean_2[['PAY_1', 'PAY_2']].info()

Os valores deste dataset são de Taiwan e o valor monetário é em doláres taiwanses

In [ ]:
sns.histplot(df_clean_2['LIMIT_BAL'], bins=11)
plt.title('Limite de crédito', fontsize=20, loc='left') 
plt.xlabel('Valores', fontsize=15, color='#696969') 
plt.ylabel('Frequência', fontsize=15, color='#696969')

In [ ]:
sns.histplot(df_clean_2['AGE'], bins=11)
plt.title('Idades', fontsize=20, loc='left') 
plt.xlabel('Valores', fontsize=15, color='#696969') 
plt.ylabel('Frequência', fontsize=15, color='#696969')

In [ ]:
df_clean_2[['LIMIT_BAL', 'AGE']].describe()

Na parte de educação, os valores 0,5 e 6 devem ficar no valor 4, que corresponde por "outros"

In [ ]:
df_clean_2['EDUCATION'].value_counts()

In [ ]:
df_clean_2.replace({'EDUCATION': {0: 4, 5: 4, 6: 4}}, inplace=True) #juntando os valores que estavam fora do dicionario de dados.
df_clean_2['EDUCATION'].value_counts()

O valor 0 não esta np dicionario de dados e deve ser realocado para o valor 3 "outros"

In [ ]:
df_clean_2['MARRIAGE'].value_counts()

In [ ]:
df_clean_2.replace({'MARRIAGE': {0: 3}}, inplace=True)
df_clean_2['MARRIAGE'].value_counts()

In [ ]:
df_clean_2.groupby('EDUCATION').agg({'default payment next month': 'mean'}).plot.bar(legend=False)

plt.title('Educação e Taxa de Inadimplência', fontsize=20, loc='left')
plt.xlabel('Nível de Educação', fontsize=15, color='#696969')
plt.ylabel('Taxa de Inadimplência', fontsize=15, color='#696969')

plt.show()

7.Implementação da OHE para uma caracteristica categorica

In [ ]:
df_clean_2['EDUCATION_CAT'] = 'none'        #OHE
df_clean_2[['EDUCATION', 'EDUCATION_CAT']].head()

Criando o dicionario para o mapeamento das categorias de instrução de Educação

In [ ]:
cat_mapping = {
    1: 'graduate school',
    2: 'university',
    3: 'high school',
    4: 'others'
}

Novo dataframe com um número igual de colunas e níveis da variável categórica

In [ ]:
df_clean_2['EDUCATION_CAT'] = df_clean_2['EDUCATION'].map(cat_mapping)
df_clean_2[['EDUCATION', 'EDUCATION_CAT']].head(10)

In [ ]:
edu_ohe = pd.get_dummies(df_clean_2['EDUCATION_CAT'])
edu_ohe.head(10)

Comparando os dois datasets se eles conferem com os valores

In [ ]:
df_with_ohe = pd.concat([df_clean_2, edu_ohe], axis=1)
df_with_ohe[['EDUCATION_CAT', 'graduate school', 'high school', 'university', 'others']].head(10)

cópia do arquivo para garantir que tenha um beckup caso ocorra algum erro no futuro

In [ ]:
df_with_ohe.to_csv('Chapter_1_cleaned_data.csv', index=False)

Explorando as caracteristicas do status de pagamento do dataset

In [ ]:
df = pd.read_csv('C:\\Users\\pedro\\OneDrive\\Ambiente de Trabalho\\Livro\\Chapter_1_cleaned_data.csv')

Juntar os históricos financeiros para serem melhores analisados

In [ ]:
pay_feats = ['PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
df[pay_feats].describe()

In [ ]:
df[pay_feats[0]].value_counts().sort_index()

In [ ]:
pay_1_bins = np.array(range(-2, 10)) - 0.5
pay_1_bins

In [ ]:
plt.rcParams['font.size'] = 4
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

for i, col in enumerate(pay_feats):
    sns.histplot(df[col], bins=pay_1_bins, ax=axes[i//3, i%3])
    axes[i//3, i%3].set_title(f'Histograma de {col}', fontsize=6)
    axes[i//3, i%3].set_xlabel('Valores', fontsize=5, color='#696969')
    axes[i//3, i%3].set_ylabel('Frequência', fontsize=5, color='#696969')

plt.show()


In [ ]:
df.loc[df['PAY_2']==2, ['PAY_2', 'PAY_3']].head()

Explorando o restante dos dados presentes no dataset

In [ ]:
bill_feats = ['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']
pay_amt_feats = ['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']

## Conclusão

Em agosto acontece um aumento milhares de contas com atraso de 2 meses de pagamento, não fazendo muito sentido, porém de acordo com a proposta do projeto, o cliente teve problemas em obter dados do último mês, o que gerou dados incorretos para contas que tem atraso de 1 mês no pagamento, sendo que foi resolvido parcialemnte esse problema em setembro.


Em toda a análise realizada, foi possível descobrir que poderiamos ter prejudicado o projeto com os dados recebidos, pois eles não eram consistentes internamente. A maioria dos meses das características de status de pagamento estava com um problema no relatório de dados, incluindo valores absurdos nos dados da instituição.